In [1]:
import numpy as np
import tensorflow as tf
from tensorflow.keras import layers
from tensorflow import keras

In [2]:
shakespeare_url = "https://homl.info/shakespeare"
filepath = keras.utils.get_file("shakespeare.txt",shakespeare_url)
filepath

1130496/1115394 [==============================] - 0s 0us/step


'/root/.keras/datasets/shakespeare.txt'

In [3]:
with open(filepath) as f:
  shakespeare_text = f.read()

In [4]:
shakespeare_text[:100]

'First Citizen:\nBefore we proceed any further, hear me speak.\n\nAll:\nSpeak, speak.\n\nFirst Citizen:\nYou'

In [5]:
type(shakespeare_text)

str

In [6]:
tokenizer = keras.preprocessing.text.Tokenizer(char_level=True)
tokenizer.fit_on_texts([shakespeare_text])

In [7]:
tokenizer.texts_to_sequences(["first"])

[[20, 6, 9, 8, 3]]

In [8]:
tokenizer.sequences_to_texts([[20,6,9,8,3]])

['f i r s t']

In [9]:
no_tokens = len(tokenizer.word_index)

In [10]:
[encoded] = np.array(tokenizer.texts_to_sequences([shakespeare_text]))-1

In [11]:
ds_size = len(encoded)
train_size = ds_size*90//100

In [12]:
train_size

1003854

In [13]:
len(tf.data.Dataset.from_tensor_slices(encoded[:train_size]))

1003854

In [14]:
ds = tf.data.Dataset.from_tensor_slices(encoded[:train_size])

In [15]:
window_size = 101
ds = ds.window(window_size,shift=1,drop_remainder=True)

In [16]:
batch_size = 32
ds = ds.flat_map(lambda window:window.batch(window_size))

In [17]:
ds = ds.shuffle(10000).batch(batch_size)

In [18]:
ds = ds.map(lambda windows:(windows[:,:-1],windows[:,1:]))

In [19]:
ds = ds.map(lambda X_batch,y_batch:(tf.one_hot(X_batch,depth=no_tokens),y_batch))

In [20]:
for X,y in ds.take(1):
  print(tf.shape(X),tf.shape(y))

tf.Tensor([ 32 100  39], shape=(3,), dtype=int32) tf.Tensor([ 32 100], shape=(2,), dtype=int32)


In [21]:
ds = ds.prefetch(1)

In [93]:
model = tf.keras.models.Sequential([
    layers.LSTM(units=128,return_sequences=True,input_shape=[None,39]),
    layers.LSTM(units=128,return_sequences=True),
    layers.LSTM(units=64,return_sequences=True),
    layers.TimeDistributed(layers.Dense(units=no_tokens,activation="softmax"))
],name="char_rnn")

In [94]:
model.compile(loss="sparse_categorical_crossentropy",
              optimizer="adam")

history = model.fit(ds,epochs=2)

Epoch 1/2
31368/31368 [==============================] - 626s 19ms/step - loss: 1.0967
Epoch 2/2
31368/31368 [==============================] - 610s 19ms/step - loss: 0.7673


In [95]:
def preprocess(text):
    X_new = np.array(tokenizer.texts_to_sequences(text))-1
    return tf.one_hot(X_new,depth=no_tokens)

In [96]:
X_new = preprocess(["How are yo"])

In [97]:
X_new.shape

TensorShape([1, 10, 39])

In [98]:
y_pred = model(X_new)

In [99]:
y_pred.shape

TensorShape([1, 10, 39])

In [100]:
temp_pred = np.argmax(y_pred,axis=-1)+1
temp_pred

array([[ 2, 14,  1,  3, 12,  3,  1, 22,  4, 14]])

In [101]:
tokenizer.sequences_to_texts(temp_pred)[0][-1]

'u'

In [102]:
def next_char(text,temperature=1):
    X_new = preprocess(text=text)
    y_proba = model(X_new)[0,-1:,:]
    rescaled_logits = tf.math.log(y_proba)/temperature
    char_id = tf.random.categorical(rescaled_logits,num_samples=1)+1
    return tokenizer.sequences_to_texts(char_id.numpy())[0]

In [103]:
def complete_text(text,n_chars=50,temperature=1):
    for _ in range(n_chars):
        text += next_char(text,temperature=temperature)
    return text

In [110]:
print(complete_text("i",temperature=1))

itooorn grnt,s o'n,'lsonooo rds onotfs,ro rltrnns n


In [ ]:
model.save("char_rnn.h5")